In [197]:
#数据导入

import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
import time
%matplotlib inline

dataset = pd.read_csv(r'E:\Datasets-2020.11.6\5.0-Site_31-TDG.csv')

dataset = dataset.rename(columns={
    u'31 TDG - Active Energy Delivered-Received (kWh)': 'AE_Power',
    u'31 TDG - Current Phase Average (A)': 'Current', #电流
    u'31 TDG - Active Power (kW)': 'Power',   #功率
    u'31 TDG - Performance Ratio (%)': 'PR',   #性能比
    u'31 TDG - Wind Speed (m/s)': 'WindSpeed',    #风速
    u'31 TDG - Weather Temperature': 'Temp', #气温
    u'31 TDG - Weather Relative Humidity (%)': 'Humidaty',   #相对湿度
    u'31 TDG - Global Horizontal Radiation': 'GHI',   #全球水平辐照度
    u'31 TDG - Diffuse Horizontal Radiation': 'DHI',  #扩散水平辐射
    u'31 TDG - Wind Direction (Degrees)': 'WD',  #风向
    u'31 TDG - Weather Daily Rainfall (mm)':'RF',#降雨
})

dataset = dataset.drop(['AE_Power','Current','PR','WindSpeed','WD','RF'],axis=1)#删除列,直接是列名

dataset = dataset.dropna(subset=['Power'])# 删除功率为空的数据组

dataset = dataset.fillna(0) # 所有NAN值赋0

dataset = dataset.drop(dataset[dataset.Power < 0].index)
dataset = dataset.drop(dataset[(dataset.Temp > 2000)|(dataset.Temp < 0)].index)
dataset = dataset.drop(dataset[dataset.Humidaty < 0].index)
dataset = dataset.drop(dataset[(dataset.GHI > 2000)|(dataset.GHI < 0)].index)
dataset = dataset.drop(dataset[dataset.DHI < 0].index)

df14 = pd.read_csv(r'E:\Datasets-2020.11.6\uv-alice-springs-2014.csv')
df15 = pd.read_csv(r'E:\Datasets-2020.11.6\uv-alice-springs-2015.csv')
df16 = pd.read_csv(r'E:\Datasets-2020.11.6\uv-alice-springs-2016.csv')
df17 = pd.read_csv(r'E:\Datasets-2020.11.6\uv-alice-springs-2017.csv')
df18 = pd.read_csv(r'E:\Datasets-2020.11.6\uv-alice-springs-2018.csv')
df19 = pd.read_csv(r'E:\Datasets-2020.11.6\uv-alice-springs-2019.csv')

df14['timestamp'] = pd.to_datetime(df14['timestamp'])
df15['timestamp'] = pd.to_datetime(df15['timestamp'])
df16['timestamp'] = pd.to_datetime(df16['timestamp'])
df17['timestamp'] = pd.to_datetime(df17['timestamp'])
df18['timestamp'] = pd.to_datetime(df18['timestamp'])
df19['timestamp'] = pd.to_datetime(df19['timestamp'])
dataset['timestamp'] = pd.to_datetime(dataset['timestamp'])

df = pd.concat([df14,df15,df16,df17,df18,df19],axis=0) #axis=0表示按行操作
df = df.drop(['Lat','Lon'],axis=1)

dataset_n = pd.merge(dataset,df,how='left',on='timestamp')

dataset_n = dataset_n.dropna(subset=['UV_Index'])
dataset_n = dataset_n.drop(dataset_n[dataset_n.UV_Index < 0].index)

minibatch=288

x_train=[]
y_train=[]
x_valid=[]
y_valid=[]
x_test=[]
y_test=[]

training_set = dataset_n.iloc[0:518688,1:7] #1441x288 # 360x288 415008:518688
test_set = dataset_n.iloc[568750:576526,1:7] # 27x288

sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(training_set)
test_set_scaled=sc.transform(test_set)

np.random.seed(6)
np.random.shuffle(x_train)
np.random.seed(6)
np.random.shuffle(y_train)
tf.random.set_seed(6)

x_train = tf.cast(training_set_scaled[0:415008,1:6], tf.float32)
y_train = tf.cast(training_set_scaled[0:415008,0:1], tf.float32)
x_valid = tf.cast(training_set_scaled[415008:518689,1:6], tf.float32)
y_valid = tf.cast(training_set_scaled[415008:518689,0:1], tf.float32)
x_test = tf.cast(test_set_scaled[:,1:6], tf.float32)
y_test = tf.cast(test_set_scaled[:,0:1], tf.float32)

# from_tensor_slices函数使输入特征和标签值一一对应。（把数据集分批次，每个批次batch组数据）
train_db = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(minibatch)
valid_db = tf.data.Dataset.from_tensor_slices((x_valid, y_valid)).batch(minibatch)

No_layer = 3 #层数3
layer_0_N=5 #输入层神经元5
layer_1_N=5 #隐藏层神经元5
layer_2_N=5 #隐藏层神经元5
layer_3_N=1 #输出层神经元1

In [198]:
#参数初始化
w1 = tf.Variable(tf.random.normal([layer_0_N, layer_1_N],mean=0, stddev=1))
b1 = tf.Variable(tf.random.normal([layer_1_N],mean=1, stddev=1))
w2 = tf.Variable(tf.random.normal([layer_1_N, layer_2_N], mean=0, stddev=1))
b2 = tf.Variable(tf.random.normal([layer_2_N], mean=1, stddev=1))
w3 = tf.Variable(tf.random.normal([layer_2_N, layer_3_N],mean=0, stddev=1))
b3 = tf.Variable(tf.random.normal([layer_3_N], mean=1, stddev=1))

In [199]:
lr =  0.1 # 学习率为0.1
train_loss_results = []  # 将每轮的loss记录在此列表中，为后续画loss曲线提供数据
test_acc = []  # 将每轮的acc记录在此列表中，为后续画acc曲线提供数据
epoch = 20  # 循环50轮
loss_all = 0  # 每轮分4个step，loss_all记录四个step生成的4个loss的和
loss_all_v = 0
m_w1,m_w2,m_w22,m_w3,m_b1,m_b2,m_b22,m_b3 = 0,0,0,0,0,0,0,0
beta = 0.9

In [ ]:
# 训练部分
now_time = time.time()  ##2##
for epoch in range(epoch):  # 数据集级别的循环，每个epoch循环一次数据集
    for step, (x_train, y_train) in enumerate(train_db):  # batch级别的循环 ，每个step循环一个batch
        with tf.GradientTape() as tape:  # with结构记录梯度信息
            h1 = tf.matmul(x_train, w1) + b1  # 神经网络乘加运算
            a1 = tf.nn.relu(h1)  # 第一层激活函数
            h2 = tf.matmul(a1,w2) + b2 # 第二层运算
            a2 = tf.nn.relu(h2) #第二层激活函数
            h3 = tf.matmul(a2,w3) + b3 #第三层运算
            a3 = tf.nn.relu(h3) # 第三层激活函数
            loss = tf.reduce_mean(tf.square(y_train-a3))  # 采用均方误差损失函数mse = mean(sum(y-out)^2)
            loss_all += loss.numpy()  # 将每个step计算出的loss累加，为后续求loss平均值提供数据，这样计算的loss更准确
        # 计算loss对各个参数的梯度
        grads = tape.gradient(loss, [w1,w2,w3,b1,b2,b3])

        # SGD+M
        
        m_w1 = beta * m_w1 + (1 - beta) * grads[0]
        m_w2 = beta * m_w2 + (1 - beta) * grads[1]
        m_w3 = beta * m_w3 + (1 - beta) * grads[2]
        m_b1 = beta * m_b1 + (1 - beta) * grads[3]
        m_b2 = beta * m_b2 + (1 - beta) * grads[4]
        m_b3 = beta * m_b3 + (1 - beta) * grads[5]
        
        w1.assign_sub(lr * m_w1)  # 参数w1自更新
        w2.assign_sub(lr * m_w2)  # 参数w2自更新
        w3.assign_sub(lr * m_w3)  # 参数w3自更新
        b1.assign_sub(lr * m_b1)  # 参数b1自更新
        b2.assign_sub(lr * m_b2)  # 参数b2自更新
        b3.assign_sub(lr * m_b3)  # 参数b3自更新
    
        h1_v = tf.matmul(x_valid, w1) + b1
        a1_v = tf.nn.relu(h1_v)
        h2_v = tf.matmul(a1_v, w2) + b2
        a2_v = tf.nn.relu(h2_v)
        h3_v = tf.matmul(a2_v, w3) + b3
        a3_v = tf.nn.relu(h3_v)
        loss_v = tf.reduce_mean(tf.square(y_valid-a3_v))

    # 每个epoch，打印loss信息
    print("Epoch {}, loss: {},loss_valid:{}".format(epoch, loss_all / (len(x_train)/minibatch), loss_v))
    train_loss_results.append(loss_all / (len(x_train)/minibatch))  # 将4个step的loss求平均记录在此变量中
    loss_all = 0  # loss_all归零，为记录下一个epoch的loss做准备
print(' train_loss_results=', train_loss_results)

Epoch 0, loss: 199.71041605994105,loss_valid:0.1386561095714569
Epoch 1, loss: 199.13108322396874,loss_valid:0.1386561095714569
Epoch 2, loss: 199.13108322396874,loss_valid:0.1386561095714569
Epoch 3, loss: 199.13108322396874,loss_valid:0.1386561095714569
Epoch 4, loss: 199.13108322396874,loss_valid:0.1386561095714569
Epoch 5, loss: 199.13108322396874,loss_valid:0.1386561095714569
Epoch 6, loss: 199.13108322396874,loss_valid:0.1386561095714569
Epoch 7, loss: 199.13108322396874,loss_valid:0.1386561095714569
Epoch 8, loss: 199.13108322396874,loss_valid:0.1386561095714569
Epoch 9, loss: 199.13108322396874,loss_valid:0.1386561095714569
Epoch 10, loss: 199.13108322396874,loss_valid:0.1386561095714569
Epoch 11, loss: 199.13108322396874,loss_valid:0.1386561095714569


In [ ]:
w1,b1

In [ ]:
pred_sum=[]

p1 = tf.matmul(x_test, w1) + b1
q1 = tf.nn.relu(p1)
p2 = tf.matmul(q1, w2) + b2
q2 = tf.nn.relu(p2)
p3 = tf.matmul(q2, w3) + b3
q3 = tf.nn.relu(p3)

test_set_p = test_set_scaled
test_set_p = np.array(test_set_p)
test_set_p[:,0:1] = q3
p_y_test = sc.inverse_transform(test_set_p)
pred=p_y_test[:,0:1]


In [ ]:
day_pred=[]
day_test=[]
for n in range(0,7776,288):
    day_pred.append(pred[n:n+288].sum())
    day_test.append(np.array(test_set)[n:n+288,0:1].sum())
print(np.array(day_pred))
print(np.array(day_test))

In [ ]:
plt.figure(figsize=(30,10))
ax = plt.axes()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.plot(day_pred,color='red',label='pre power')
plt.plot(np.array(day_test),color='blue',label='test power')
plt.title('Pre/Test by day',fontsize=28,fontweight='heavy', pad=40)
plt.xlabel('Days',fontsize=18)
plt.ylabel('Power(kW)',fontsize=18)
plt.legend(fontsize=18)
plt.show()

In [ ]:
np.array(day_test)-np.array(day_pred)